In [462]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
import glob, os, shutil
from matplotlib import pyplot as plt

## 1) Scraping Election Data

Election data is scraped from Turkish high election board's database, using Selenium package. Making a query in the database consists of 3 steps: 1) Query Options ("Sorgulama Secenekleri"), 2) Select Election Name ("Secim Adi Seciniz"), and 3) Election Results ("Secim Sonuclari"). 

Once the final step is reached, three dropdown lists (for filtering city, district and neighborhood name), and a filter for ballot box id appear. We are interested in neighborhood-wise election results. (Note: District is the subdivision of city, and neighborhood is the subdivision of district)

Due to database's design, when only a city is selected (but not a district) district-wise election results are shown
When both a city and district are selected, results of each ballot box (in the district) are shown. Hence, we will scrape the results by selecting both the city and district name, and then aggregating the ballot box results by neighborhood names.

In [26]:
database_url = 'https://sonuc.ysk.gov.tr'
istanbul = 'İSTANBUL'

# Election names
ist_reelect = 'İSTANBUL BÜYÜKŞEHİR BELEDİYE BAŞKANLIĞI YENİLEME SEÇİMİ (23/06/2019)'
loc_2019 = '31 MART 2019 MAHALLİ İDARELER GENEL SEÇİMİ (31/03/2019)'
gen_2018 = 'CUMHURBAŞKANI VE 27.DÖNEM MİLLETVEKİLİ GENEL SEÇİMİ (24/06/2018)'
ref_2017 = '2017 HALKOYLAMASI (16/04/2017)'
gen_nov_2015 = '26.DÖNEM MİLLETVEKİLİ GENEL SEÇİMİ (01/11/2015)'
gen_jun_2015 = '25.DÖNEM MİLLETVEKİLİ GENEL SEÇİMİ (07/06/2015)'

In [27]:
# Creates a webdriver object for Chrome and navigates to Turkish high election board's election results database
chrome = webdriver.Chrome(executable_path='/Users/ercansen/Desktop/apps/chromedriver')
chrome.get(database_url)

In [28]:
# Website prompts the user to pick how to query the results; query by election name (third button) is selected
byElecNm_classname = 'ma-0.pa-0.border-0.cursor-pointer.w-100'

buttons_intro = list()
buttons_intro = chrome.find_elements_by_class_name(byElecNm_classname)
button_byElecNm = buttons_intro[2]
button_byElecNm.click()

In [29]:
# To open up the dropdown list of past 11 elections in Turkey, scans the corresponding arrow
elec_arrow_xpath = "//*[@id='collapsePanelThree']/div[1]/div/div/form/div[1]/div/div/ng-select/div/span"
elec_arrow = chrome.find_element_by_xpath(elec_arrow_xpath)

### Istanbul 2019 Mayoral Rerun Election

In [6]:
# Clicks elec_arrow so that the dropdown list is opened; desired election is selected from the list
elec_arrow.click()

for option in chrome.find_elements_by_class_name('ng-option'):
    if option.text == ist_reelect:
        option.click()
        break

In [7]:
# Clicks the green 'continue' ("Devam Et") button
continue_classname = 'btn.btn-success.rounded-0.mr-5'

button_continue = chrome.find_element_by_class_name(continue_classname)
button_continue.click()

In [8]:
# Clicks the corresponding arrow for city dropdown list
city_arrow_xpath = "//*[@id='collapsePanelFour']/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/div/span"
city_arrow = chrome.find_element_by_xpath(city_arrow_xpath)
city_arrow.click()

In [9]:
# Selects Istanbul from the list of cities (although it is the only option for the rerun mayoral election)
city_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/ng-dropdown-panel/div'
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul:
        option.click()
        break

In [18]:
# Scans the corresponding arrow for district dropdown list
district_arrow_xpath = "//*[@id='collapsePanelFour']/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/span"
district_arrow = chrome.find_element_by_xpath(district_arrow_xpath)

In [19]:
district_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/ng-dropdown-panel/div'
query_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[2]/div/div/button'
district_input_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/div/div[2]/input'
save_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/app-yurtici-muhtarlik-listesi/fieldset/div[2]/div/button'
accept_button_xpath = '/html/body/ngb-modal-window/div/div/div[2]/div/button[1]'

query_button = chrome.find_element_by_xpath(query_button_xpath)
district_input = chrome.find_element_by_xpath(district_input_xpath)

# Retrieves a list of districts of Istanbul
districts = list()
district_arrow.click()
for option in chrome.find_element_by_xpath(district_dropdown_xpath).find_elements_by_class_name('ng-option'):
    districts.append(option.text)

# For each district in the list, corresponding box receives district input, 
## navigates to election results and downloads them
for i in range(len(districts)):
    district = districts[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)   

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ngb-modal-window/div/div/div[2]/div/button[1]"}
  (Session info: chrome=83.0.4103.116)


In [13]:
# Creates directories in desired location in computer for storing the output excel files
downloads  = '/Users/ercansen/Downloads/'
dest_dir_1 = '/Users/ercansen/Desktop/apps/istanbul/2019-06' 
dest_dir_2 = '/Users/ercansen/Desktop/apps/istanbul/2019-06/raw'
os.mkdir(dest_dir_1)
os.mkdir(dest_dir_2)

# Queries all excel files with election results using glob
files = glob.glob(downloads + 'Yurt*.xlsx')

# Copies Excel files to the directory that was just created
for file in files:
    if os.path.isfile(file) and len(files) == 39:
        shutil.copy2(file, dest_dir_2)
        
# Deletes original files from 'Downloads' after copying
for file in files:
    if os.path.isfile(file) and len(files) == 39:
        os.remove(file)

### 2019 Local Election (cancelled)

In [14]:
# To select another election, we go back a step by clicking on 
## corresponding arrow for "Select Election Name"
elect_reselect_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[1]/div/div[2]/a/span'
chrome.find_element_by_xpath(elect_reselect_xpath).click()

# We make a new selection for election name
elec_arrow.click()
for option in chrome.find_elements_by_class_name('ng-option'):
    if option.text == loc_2019:
        option.click()
        break

In [19]:
# Scans election type arrow
elec_type_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[1]/div[2]/div/ng-select/div/span'
chrome.find_element_by_xpath(elec_type_arrow_xpath).click()

In [20]:
metro_municip_elec = 'BÜYÜKŞEHİR BELEDİYE BAŞKANLIĞI SEÇİMLERİ'
elec_type_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[1]/div[2]/div/ng-select/ng-dropdown-panel'
continue_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[2]/div/div/button[1]'

# At local elections, multiple elections are carried out (metropolitan mayor, district mayor, town council)
# We are interested in metro. mayor, since it is most suitable for our purposes, hence select that on dropdown list
elec_type_dropdown = chrome.find_element_by_xpath(elec_type_dropdown_xpath)

for option in elec_type_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == metro_municip_elec:
        option.click()
        break
        
time.sleep(5)
chrome.find_element_by_xpath(continue_xpath).click()

In [21]:
# Opens up the dropdown list for city name
city_arrow_xpath = "//*[@id='collapsePanelFour']/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/div/span"
city_arrow = chrome.find_element_by_xpath(city_arrow_xpath)
city_arrow.click()

In [22]:
# Selects Istanbul from the list of cities
city_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/ng-dropdown-panel/div'
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul:
        option.click()
        break

In [24]:
query_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[2]/div/div/button'
district_input_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/div/div[2]/input'
save_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/app-yurtici-muhtarlik-listesi/fieldset/div[2]/div/button'
accept_button_xpath = '/html/body/ngb-modal-window/div/div/div[2]/div/button[1]'

query_button = chrome.find_element_by_xpath(query_button_xpath)
district_input = chrome.find_element_by_xpath(district_input_xpath)

# We already have the list 'districts' from previous step
# Enters each district name to corresponding input box and downloads the election results
for i in range(len(districts)):
    district = districts[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    chrome.find_element_by_xpath(query_button_xpath).click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  

In [25]:
# Creates directories, copies .xlsx files to them and deletes the originals from Downloads 
downloads  = '/Users/ercansen/Downloads/'
dest_dir_1 = '/Users/ercansen/Desktop/apps/istanbul/2019-03' 
dest_dir_2 = '/Users/ercansen/Desktop/apps/istanbul/2019-03/raw'
os.mkdir(dest_dir_1)
os.mkdir(dest_dir_2)

files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files) == 39:
        os.remove(file)

### 2018 General Election (Presidential)

In [10]:
# Selects the new election name
elect_reselect_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[1]/div/div[2]/a/span'
chrome.find_element_by_xpath(elect_reselect_xpath).click()
elec_arrow.click()

for option in chrome.find_elements_by_class_name('ng-option'):
    if option.text == gen_2018:
        option.click()
        break

In [11]:
# Scans the arrow for election type
elec_type_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[1]/div[2]/div/ng-select/div/span'
chrome.find_element_by_xpath(elec_type_arrow_xpath).click()

In [12]:
# Selects the election type -- presidential
presidential = 'CUMHURBAŞKANI SEÇİMİ'
elec_type_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[1]/div[2]/div/ng-select/ng-dropdown-panel'
elec_type_dropdown = chrome.find_element_by_xpath(elec_type_dropdown_xpath)

time.sleep(3)

for option in elec_type_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == presidential:
        option.click()
        break

In [14]:
# Continue
continue_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[2]/div/div/button[1]'
chrome.find_element_by_xpath(continue_xpath).click()

In [15]:
# Navigates to domestic results, as opposed to foreign (expat voters)
domestic_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[1]/div[1]/div[2]/label/input'
chrome.find_element_by_xpath(domestic_xpath).click()

In [16]:
# Selects Istanbul from the list of cities
city_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/div/span'
city_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/ng-dropdown-panel'

chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul:
        option.click()
        break

In [20]:
query_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[2]/div/div/button'
district_input_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/div/div[2]/input'
save_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/app-yurtici-muhtarlik-listesi/fieldset/div[2]/div/button'
accept_button_xpath = '/html/body/ngb-modal-window/div/div/div[2]/div/button[1]'

query_button = chrome.find_element_by_xpath(query_button_xpath)
district_input = chrome.find_element_by_xpath(district_input_xpath)

# We already have the list 'districts' from an earlier step
# Enters each district name to input box and downloads the election results
for i in range(len(districts)):
    district = districts[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)

In [24]:
# Creates directories, copies .xlsx files to them and deletes the originals from Downloads 
downloads  = '/Users/ercansen/Downloads/'
dest_dir_1 = '/Users/ercansen/Desktop/apps/istanbul/2018-06-PRES' 
dest_dir_2 = '/Users/ercansen/Desktop/apps/istanbul/2018-06-PRES/raw'
os.mkdir(dest_dir_1)
os.mkdir(dest_dir_2)

files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files) == 39:
        os.remove(file)

### 2018 General Election (MP)

In [32]:
# Selects the new election name
elect_reselect_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[1]/div/div[2]/a/span'
chrome.find_element_by_xpath(elect_reselect_xpath).click()
elec_arrow.click()

for option in chrome.find_elements_by_class_name('ng-option'):
    if option.text == gen_2018:
        option.click()
        break

In [33]:
# Scans the arrow for election type
elec_type_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[1]/div[2]/div/ng-select/div/span'
chrome.find_element_by_xpath(elec_type_arrow_xpath).click()

In [34]:
# Selects the election type -- member of parliament
mp = 'MİLLETVEKİLİ GENEL SEÇİMİ'
elec_type_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[1]/div[2]/div/ng-select/ng-dropdown-panel'
elec_type_dropdown = chrome.find_element_by_xpath(elec_type_dropdown_xpath)

time.sleep(3)

for option in elec_type_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == mp:
        option.click()
        break

In [35]:
# Continue
continue_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[2]/div/div/button[1]'
chrome.find_element_by_xpath(continue_xpath).click()

In [36]:
# Navigates to domestic results, as opposed to foreign (expat voters)
domestic_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[1]/div[1]/div[2]/label/input'
chrome.find_element_by_xpath(domestic_xpath).click()

In [37]:
# Selects Istanbul from the list of cities
city_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/div/span'
city_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/ng-dropdown-panel'

# Istanbul is divided into 3 regions for MP elections, all of which have to be scraped separately
istanbul_1 = 'İSTANBUL-1'
istanbul_2 = 'İSTANBUL-2'
istanbul_3 = 'İSTANBUL-3'

chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul_1:
        option.click()
        break

In [38]:
district_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/span'
query_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[2]/div/div/button'
district_input_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/div/div[2]/input'
save_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/app-yurtici-muhtarlik-listesi/fieldset/div[2]/div/button'
accept_button_xpath = '/html/body/ngb-modal-window/div/div/div[2]/div/button[1]'

query_button = chrome.find_element_by_xpath(query_button_xpath)
district_input = chrome.find_element_by_xpath(district_input_xpath)

# Retrieves Istanbul-1 MP region's district list
districts_1 = list()
chrome.find_element_by_xpath(district_arrow_xpath).click()
for option in chrome.find_element_by_xpath(district_dropdown_xpath).find_elements_by_class_name('ng-option'):
    districts_1.append(option.text)

# Enters each district name to input box and downloads the election results
for i in range(len(districts_1)):
    district = districts_1[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  

In [39]:
# Creates directories, copies .xlsx files to them and deletes the originals from Downloads 
downloads  = '/Users/ercansen/Downloads/'
dest_dir_1 = '/Users/ercansen/Desktop/apps/istanbul/2018-06-MP' 
dest_dir_2 = '/Users/ercansen/Desktop/apps/istanbul/2018-06-MP/raw'
os.mkdir(dest_dir_1)
os.mkdir(dest_dir_2)

files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files):
        os.remove(file)

In [40]:
# Istanbul's 2nd electoral region
chrome.find_element_by_xpath(city_arrow_xpath).click()
chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul_2:
        option.click()
        break

In [41]:
# Retrieves Istanbul-2 MP region's district list
districts_2 = list()
chrome.find_element_by_xpath(district_arrow_xpath).click()
for option in chrome.find_element_by_xpath(district_dropdown_xpath).find_elements_by_class_name('ng-option'):
    districts_2.append(option.text)

# Enters each district name to input box and downloads the election results
for i in range(len(districts_2)):
    district = districts_2[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  

In [42]:
files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files):
        os.remove(file)

In [43]:
# Istanbul's 3rd electoral region
chrome.find_element_by_xpath(city_arrow_xpath).click()
chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul_3:
        option.click()
        break

In [44]:
# Retrieves Istanbul-3 MP region's district list
districts_3 = list()
chrome.find_element_by_xpath(district_arrow_xpath).click()
for option in chrome.find_element_by_xpath(district_dropdown_xpath).find_elements_by_class_name('ng-option'):
    districts_3.append(option.text)

# Enters each district name to input box and downloads the election results
for i in range(len(districts_3)):
    district = districts_3[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  

In [45]:
files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files):
        os.remove(file)

### 2017 Constitutional Referendum

In [69]:
# Selects the new election name
elect_reselect_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[1]/div/div[2]/a/span'
chrome.find_element_by_xpath(elect_reselect_xpath).click()
elec_arrow.click()

for option in chrome.find_elements_by_class_name('ng-option'):
    if option.text == ref_2017:
        option.click()
        break

In [70]:
# Continue
continue_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[2]/div/div/button[1]'
chrome.find_element_by_xpath(continue_xpath).click()

In [71]:
# Navigates to domestic results, as opposed to foreign (expat voters)
domestic_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[1]/div[1]/div[2]/label/input'
chrome.find_element_by_xpath(domestic_xpath).click()

In [72]:
# Selects Istanbul from the list of cities
city_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/div/span'
city_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/ng-dropdown-panel'

chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul:
        option.click()
        break

In [73]:
query_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[2]/div/div/button'
district_input_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/div/div[2]/input'
save_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/app-yurtici-muhtarlik-listesi/fieldset/div[2]/div/button'
accept_button_xpath = '/html/body/ngb-modal-window/div/div/div[2]/div/button[1]'

query_button = chrome.find_element_by_xpath(query_button_xpath)
district_input = chrome.find_element_by_xpath(district_input_xpath)

# We already have the list 'districts' from an earlier step
# Enters each district name to input box and downloads the election results
for i in range(len(districts)):
    district = districts[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)

In [74]:
# Creates directories, copies .xlsx files to them and deletes the originals from Downloads 
downloads  = '/Users/ercansen/Downloads/'
dest_dir_1 = '/Users/ercansen/Desktop/apps/istanbul/2017-04' 
dest_dir_2 = '/Users/ercansen/Desktop/apps/istanbul/2017-04/raw'
os.mkdir(dest_dir_1)
os.mkdir(dest_dir_2)

files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files):
        os.remove(file)

### 2015 November General Election

In [75]:
# Selects the new election name
elect_reselect_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[1]/div/div[2]/a/span'
chrome.find_element_by_xpath(elect_reselect_xpath).click()
elec_arrow.click()

for option in chrome.find_elements_by_class_name('ng-option'):
    if option.text == gen_nov_2015:
        option.click()
        break

In [77]:
# Navigates to domestic results, as opposed to foreign (expat voters)
domestic_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[1]/div[2]/div/div[2]/label/input'
chrome.find_element_by_xpath(domestic_xpath).click()

In [78]:
# Continue
continue_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[2]/div/div/button[1]'
chrome.find_element_by_xpath(continue_xpath).click()

In [79]:
city_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/div/span'
city_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/ng-dropdown-panel'

# Istanbul is divided into 3 regions for MP elections, all of which have to be scraped separately
istanbul_1 = 'İSTANBUL-1'
istanbul_2 = 'İSTANBUL-2'
istanbul_3 = 'İSTANBUL-3'

chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul_1:
        option.click()
        break

In [82]:
# Istanbul's 1st MP district
district_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/span'
query_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[2]/div/div/button'
district_input_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/div/div[2]/input'
save_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/app-yurtici-muhtarlik-listesi/fieldset/div[2]/div/button'
accept_button_xpath = '/html/body/ngb-modal-window/div/div/div[2]/div/button[1]'

query_button = chrome.find_element_by_xpath(query_button_xpath)
district_input = chrome.find_element_by_xpath(district_input_xpath)

# Enters each district name to input box and downloads the election results
for i in range(len(districts_1)):
    district = districts_1[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  

In [83]:
# Creates directories, copies .xlsx files to them and deletes the originals from Downloads 
downloads  = '/Users/ercansen/Downloads/'
dest_dir_1 = '/Users/ercansen/Desktop/apps/istanbul/2015-11' 
dest_dir_2 = '/Users/ercansen/Desktop/apps/istanbul/2015-11/raw'
os.mkdir(dest_dir_1)
os.mkdir(dest_dir_2)

files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files):
        os.remove(file)

In [84]:
# Istanbul's 2nd electoral region
chrome.find_element_by_xpath(city_arrow_xpath).click()
chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul_2:
        option.click()
        break

In [85]:
# Enters each district name to input box and downloads the election results
for i in range(len(districts_2)):
    district = districts_2[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  

In [86]:
files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files):
        os.remove(file)

In [87]:
# Istanbul's 3rd electoral region
chrome.find_element_by_xpath(city_arrow_xpath).click()
chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul_3:
        option.click()
        break

In [88]:
# Enters each district name to input box and downloads the election results
for i in range(len(districts_3)):
    district = districts_3[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  

In [89]:
files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files):
        os.remove(file)

### 2015 June General Election

In [90]:
# Selects the new election name
elect_reselect_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[1]/div/div[2]/a/span'
chrome.find_element_by_xpath(elect_reselect_xpath).click()
elec_arrow.click()

for option in chrome.find_elements_by_class_name('ng-option'):
    if option.text == gen_jun_2015:
        option.click()
        break

In [91]:
# Navigates to domestic results, as opposed to foreign (expat voters)
domestic_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[1]/div[2]/div/div[2]/label/input'
chrome.find_element_by_xpath(domestic_xpath).click()

In [92]:
# Continue
continue_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sorgu/div/div/div[2]/div[1]/div/div/form/div[2]/div/div/button[1]'
chrome.find_element_by_xpath(continue_xpath).click()

In [93]:
city_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/div/span'
city_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/ng-dropdown-panel'

# Istanbul is divided into 3 regions for MP elections, all of which have to be scraped separately
istanbul_1 = 'İSTANBUL-1'
istanbul_2 = 'İSTANBUL-2'
istanbul_3 = 'İSTANBUL-3'

chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul_1:
        option.click()
        break

In [94]:
# Istanbul's 1st MP district
district_arrow_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/span'
query_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[2]/div/div/button'
district_input_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/div/div[2]/input'
save_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/app-yurtici-muhtarlik-listesi/fieldset/div[2]/div/button'
accept_button_xpath = '/html/body/ngb-modal-window/div/div/div[2]/div/button[1]'

query_button = chrome.find_element_by_xpath(query_button_xpath)
district_input = chrome.find_element_by_xpath(district_input_xpath)

# Enters each district name to input box and downloads the election results
for i in range(len(districts_1)):
    district = districts_1[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  

In [95]:
# Creates directories, copies .xlsx files to them and deletes the originals from Downloads 
downloads  = '/Users/ercansen/Downloads/'
dest_dir_1 = '/Users/ercansen/Desktop/apps/istanbul/2015-06' 
dest_dir_2 = '/Users/ercansen/Desktop/apps/istanbul/2015-06/raw'
os.mkdir(dest_dir_1)
os.mkdir(dest_dir_2)

files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files):
        os.remove(file)

In [96]:
# Istanbul's 2nd electoral region
chrome.find_element_by_xpath(city_arrow_xpath).click()
chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul_2:
        option.click()
        break

In [97]:
# Enters each district name to input box and downloads the election results
for i in range(len(districts_2)):
    district = districts_2[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  

In [98]:
files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files):
        os.remove(file)

In [99]:
# Istanbul's 3rd electoral region
chrome.find_element_by_xpath(city_arrow_xpath).click()
chrome.find_element_by_xpath(city_arrow_xpath).click()
time.sleep(3)
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul_3:
        option.click()
        break

In [100]:
# Enters each district name to input box and downloads the election results
for i in range(len(districts_3)):
    district = districts_3[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  

In [3]:
files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir_2)
        
for file in files:
    if os.path.isfile(file) and len(files):
        os.remove(file)

## 2) Aggregating Ballot Box Results with SQL

Once web scraping with Selenium is complete, we have 39 Excel spreadsheets (per 39 districts of Istanbul) for each of the 7 elections we are working with. As stated previously, these spreadsheets are at the level of a ballot box. Each ballot box holds the vote of approximately 300 voters. The neighborhood with most ballot boxes has 191, while some neighborhoods (some of which had been officially villages before the law [1] changed) consist of a single ballot box. Before any aggregation is applied, these 39 spreadsheets are merged for each election and we end up with 7 csv files that include the results of all the ballot boxes in the city, for corresponding election.

Next, a series of SQL queries are applied on the csv files that are created. First, each district is assigned a unique id, in the range 1-39. Then, these neighborhood id's are multiplied by 1000; this way, each district can 'hold' up to 99 neighborhoods, which is more than the maximum (The maximum number of nbhds in a district is 62 in Sile). For instance, 37th district can have corresponding neighborhood id's in the range 3701-3799. Using this method, every neighborhood also gets a unique id. After that, the ballot box results are aggregated by neighborhood. Standard features like vote share of a party/candidate, or turnout percentage are calculated and added as columns. In addition, feature engineering has been done to find out meaningful indicators that showcases political differences from neighborhood to neighborhood. Finally, the resulting tables are outputed into csv's. The SQL queries that result in this output can be seen at 'sql_queries.txt' that is in this github directory.

In [74]:
path_project = '/Users/ercansen/Desktop/apps/istanbul'
list_directories = [('/' + d.name) for d in os.scandir(path_project) if d.is_dir() and d.name[0] != '.']
raw = '/raw/'

In [75]:
cols_to_keep = ['İlçe Adı', 'Mahalle/Köy', 'Kayıtlı Seçmen Sayısı', 'Oy Kullanan Seçmen Sayısı', 'Toplam Geçerli Oy', 
                'AK PARTİ', 'CHP', 'MHP', 'İYİ PARTİ', 'HDP', 'SAADET', 'CUMHUR İTTİFAKI', 'MİLLET İTTİFAKI', 
                'RECEP TAYYİP ERDOĞAN', 'MUHARREM İNCE', 'MERAL AKŞENER', 'SELAHATTİN DEMİRTAŞ', 'TEMEL KARAMOLLAOĞLU',
                'EVET', 'HAYIR', 
                'HAKAN ŞÜKÜR', 'ALİ FUAT YILMAZER', 'YAKUB SAYGILI']

In [76]:
cols_dict = {'İlçe Adı':'district', 'Mahalle/Köy':'neighborhood',
             'Kayıtlı Seçmen Sayısı':'registered', 'Oy Kullanan Seçmen Sayısı':'voted', 'Toplam Geçerli Oy':'valid', 
             'AK PARTİ':'akp', 'CHP':'chp', 'MHP':'mhp', 'İYİ PARTİ':'iyip', 'HDP':'hdp', 'SAADET':'sp', 
             'CUMHUR İTTİFAKI':'cumhur', 'MİLLET İTTİFAKI':'millet', 
             'RECEP TAYYİP ERDOĞAN':'rte', 'MUHARREM İNCE':'ince', 'MERAL AKŞENER':'aksener', 
             'SELAHATTİN DEMİRTAŞ':'selo', 'TEMEL KARAMOLLAOĞLU':'karamolla',
             'EVET':'yes', 'HAYIR':'no', 
             'HAKAN ŞÜKÜR':'sukur', 'ALİ FUAT YILMAZER':'yilmazer', 'YAKUB SAYGILI':'saygili'}

In [95]:
# Loops over all folders with raw data
for i in range(len(list_directories)):
    path_to_dir    = path_project + list_directories[i] + raw
    list_raw_files = glob.glob(path_to_dir + 'Yurt*.xlsx')
    dfs = list()
    # Loops over all raw data Excel worksheets in the selected folder and reads them as dataframes
    for j in range(len(list_raw_files)):
        df = pd.read_excel(list_raw_files[j], skiprows=range(10))
        
        # Keeps only the columns of data we are interested in
        intersect = [it for it in cols_to_keep if it in df.columns]
        df = df[intersect]
        
        # Replaces districts with non-English characters with their English counterparts, as csv cannot recognize
        if df.iloc[0]['İlçe Adı'] == 'ARNAVUTKÖY':
            df['İlçe Adı'] = 'ARNAVUTKOY'
        elif df.iloc[0]['İlçe Adı'] == 'ATAŞEHİR':
            df['İlçe Adı'] = 'ATASEHIR'
        elif df.iloc[0]['İlçe Adı'] == 'BAĞCILAR':
            df['İlçe Adı'] = 'BAGCILAR'
        elif df.iloc[0]['İlçe Adı'] == 'BAHÇELİEVLER':
            df['İlçe Adı'] = 'BAHCELIEVLER'
        elif df.iloc[0]['İlçe Adı'] == 'BAKIRKÖY':
            df['İlçe Adı'] = 'BAKIRKOY'
        elif df.iloc[0]['İlçe Adı'] == 'BAŞAKŞEHİR':
            df['İlçe Adı'] = 'BASAKSEHIR'
        elif df.iloc[0]['İlçe Adı'] == 'BAYRAMPAŞA':
            df['İlçe Adı'] = 'BAYRAMPASA'
        elif df.iloc[0]['İlçe Adı'] == 'BEŞİKTAŞ':
            df['İlçe Adı'] = 'BESIKTAS'
        elif df.iloc[0]['İlçe Adı'] == 'BEYLİKDÜZÜ':
            df['İlçe Adı'] = 'BEYLIKDUZU'
        elif df.iloc[0]['İlçe Adı'] == 'BEYOĞLU':
            df['İlçe Adı'] = 'BEYOGLU'
        elif df.iloc[0]['İlçe Adı'] == 'BÜYÜKÇEKMECE':
            df['İlçe Adı'] = 'BUYUKCEKMECE'
        elif df.iloc[0]['İlçe Adı'] == 'ÇATALCA':
            df['İlçe Adı'] = 'CATALCA'
        elif df.iloc[0]['İlçe Adı'] == 'ÇEKMEKÖY':
            df['İlçe Adı'] = 'CEKMEKOY'
        elif df.iloc[0]['İlçe Adı'] == 'EYÜPSULTAN':
            df['İlçe Adı'] = 'EYUPSULTAN'
        elif df.iloc[0]['İlçe Adı'] == 'FATİH':
            df['İlçe Adı'] = 'FATIH'
        elif df.iloc[0]['İlçe Adı'] == 'GAZİOSMANPAŞA':
            df['İlçe Adı'] = 'GAZIOSMANPASA'
        elif df.iloc[0]['İlçe Adı'] == 'GÜNGÖREN':
            df['İlçe Adı'] = 'GUNGOREN'
        elif df.iloc[0]['İlçe Adı'] == 'KADIKÖY':
            df['İlçe Adı'] = 'KADIKOY'
        elif df.iloc[0]['İlçe Adı'] == 'KAĞITHANE':
            df['İlçe Adı'] = 'KAGITHANE'
        elif df.iloc[0]['İlçe Adı'] == 'KÜÇÜKÇEKMECE':
            df['İlçe Adı'] = 'KUCUKCEKMECE'
        elif df.iloc[0]['İlçe Adı'] == 'PENDİK':
            df['İlçe Adı'] = 'PENDIK'
        elif df.iloc[0]['İlçe Adı'] == 'SİLİVRİ':
            df['İlçe Adı'] = 'SILIVRI'
        elif df.iloc[0]['İlçe Adı'] == 'SULTANBEYLİ':
            df['İlçe Adı'] = 'SULTANBEYLI'
        elif df.iloc[0]['İlçe Adı'] == 'SULTANGAZİ':
            df['İlçe Adı'] = 'SULTANGAZI'
        elif df.iloc[0]['İlçe Adı'] == 'ŞİLE':
            df['İlçe Adı'] = 'SILE'
        elif df.iloc[0]['İlçe Adı'] == 'ŞİŞLİ':
            df['İlçe Adı'] = 'SISLI'
        elif df.iloc[0]['İlçe Adı'] == 'ÜMRANİYE':
            df['İlçe Adı'] = 'UMRANIYE'
        elif df.iloc[0]['İlçe Adı'] == 'ÜSKÜDAR':
            df['İlçe Adı'] = 'USKUDAR'
        elif df.iloc[0]['İlçe Adı'] == 'ZEYTİNBURNU':
            df['İlçe Adı'] = 'ZEYTINBURNU'
     
        # Inserts (partially) cleaned df to the list dfs
        dfs.append(df)
    
    # Merges all dataframes in the list dfs
    merged = pd.concat(dfs, sort=False, ignore_index=True)
    
    # Renames column names
    common_keys = cols_dict.keys() & merged.columns
    dict_intersect = {k:cols_dict[k] for k in common_keys}
    merged = merged.rename(columns=dict_intersect)
    
    # Replaces non-English characters with their English counterparts, since csv type cannot recognize them
    merged['neighborhood'] = merged['neighborhood'].str.replace('Ç','C').str.replace('İ','I').str.replace('Ş','S').str.replace('Ğ','G').str.replace('Ü','U').str.replace('Ö','O')

    # Merges 3 Gulenist candidates from 2015 June general election into a single column
    if 'sukur' in merged.columns:
        merged['sukur'] = merged['sukur'].fillna(0)
        merged['yilmazer'] = merged['yilmazer'].fillna(0)
        merged['saygili'] = merged['saygili'].fillna(0)
        merged['gulenist'] = merged.sukur + merged.yilmazer + merged.saygili
        merged = merged.drop(columns=['sukur','yilmazer','saygili'])
    
    # Saves the resulting df as a csv file
    merged.to_csv(path_project + list_directories[i] + '/' + list_directories[i] + '.csv', index=False)

## 3) Cleaning Socioeconomical Data

Unfortunately, Turkish government does not release neighborhood-wise socioeconomical data (i.e. there is nothing like US Census, although less detailed data exists). Therefore, the desired data has been received from Nezih Onur Kuru, a PhD candidate in Koc University. 

The socioconomical dataset needs some cleaning, as it has some missing values and some columns that will not be necessary for our purposes. Pandas will be used for cleaning.

In [2]:
socio = pd.read_excel('/Users/ercansen/Desktop/apps/istanbul/MAHALLE.xlsx')
socio.head()

,DEGISKEN,id,2016 Toplam Nufus,2016 Toplam Nufus (Erkek),2016 Toplam Nufus (Kadin),2015 Toplam Nufus,2015 Toplam Nufus (Erkek),2015 Toplam Nufus (Kadin),2014 Toplam Nufus,2014 Toplam Nufus (Erkek),...,BTP_2014_metropol,VATANPARTİSİ_2014_metropol,HDP_2014_metropol,CHP_2014_metropol,HEPAR_2014_metropol,MHP_2014_metropol,independents_2014_metropol,akp_2014_metropol,margin_1to2_2014_metropol,akp_margin_2014_metropol
0,ADALAR BURGAZADA,1651564d,1422,696,726,1436.0,716,720,1464,727,...,0.000000,0.003806,0.036156,0.705994,0.000951,0.010466,0.002854,second,0.475737,-0.475737
1,ADALAR HEYBELIADA,65612,3757,1954,1803,4785.0,3018,1767,4885,3078,...,0.000000,0.004026,0.028986,0.566023,0.002013,0.033816,0.002818,second,0.215781,-0.215781
2,ADALAR KINALIADA,15616684,1905,995,910,2047.0,1055,992,2322,1199,...,0.000000,0.001535,0.035806,0.612276,0.002046,0.012788,0.000512,second,0.284399,-0.284399
3,ADALAR MADEN,464d4646d46,4335,2225,2110,4356.0,2226,2130,4345,2196,...,0.001023,0.001023,0.052183,0.548772,0.001023,0.030014,0.001023,second,0.194407,-0.194407
4,ADALAR NIZAM,58,3059,1550,1509,2999.0,1534,1465,3036,1549,...,0.000000,0.002498,0.058941,0.509491,0.002498,0.030470,0.000999,second,0.124376,-0.124376


In [3]:
# Columns other than total population have some missing values for valid neighborhoods, so popl. is used to filter
socio = socio[socio['2016 Toplam Nufus'].notnull()]
socio = socio.reset_index()

In [4]:
# Splits original "district neighborhood" column into two columns of a temp df
splitting = socio['DEGISKEN'].str.split(n=1, expand=True)

In [5]:
# Adds the splitted columns back into original df; drops unnecessary columns
socio['district'] = splitting[0]
socio['nbhd'] = splitting[1]
socio = socio.drop(columns=['index','DEGISKEN'])

In [6]:
# Imports the csv with SQL-generated neighborhoood id.s
nbhds = pd.read_csv('/Users/ercansen/Desktop/apps/istanbul/sql outputs/neighborhoods.csv')
nbhds.head()

,district,neighborhood,dist_id,nbhd_id
0,ADALAR,BURGAZADA MAH.,100,101
1,ADALAR,HEYBELIADA MAH.,100,102
2,ADALAR,KINALIADA MAH.,100,103
3,ADALAR,MADEN MAH.,100,104
4,ADALAR,NIZAM MAH.,100,105


In [7]:
# Gets rid of 'MAH.' (abbreviation of neighborhood, in Turkish) from neighborhood names and strips from trailing space
nbhds['neighborhood'] = nbhds['neighborhood'].str[:-4]
nbhds['neighborhood'] = nbhds['neighborhood'].str.rstrip()

In [8]:
# Determines which of the neighborhood names are different than our clean nbhds df (scraped from govermental database)
tups = list()

for dist in nbhds['district'].unique():
    cleans = list(nbhds[nbhds.district == dist]['neighborhood'])
    raws = list(socio[socio.district == dist]['nbhd'])
    name_dif = [nb for nb in raws if nb not in cleans]
    if len(name_dif) > 0:
        print('Neighborhoods',name_dif,'in',dist,'are named differently than our clean database')
        
        for n in name_dif:
            tups.append((n,dist))

Neighborhoods ['ATATURK'] in ARNAVUTKOY are named differently than our clean database
Neighborhoods ['ORTAMAHALLE'] in BAYRAMPASA are named differently than our clean database
Neighborhoods ['MIMAR SINAN'] in BUYUKCEKMECE are named differently than our clean database
Neighborhoods ['BAHCEKOY YENIMAHALLE', 'RUMELI HISARI', 'RUMELI KAVAGI', 'SARIYER YENIMAHALLE'] in SARIYER are named differently than our clean database
Neighborhoods ['KUCUKCAMLICA'] in USKUDAR are named differently than our clean database


In [13]:
# Corrects the inconsistent names
tups_corrected = tups.copy()
corrects = ['MERKEZ ATATURK', 'ORTA', 'MIMARSINAN', 'BAHCEKOY YENI', 'RUMELIHISARI', 'RUMELIKAVAGI', 'YENI', 'KUCUK CAMLICA']

for i in range(len(corrects)):
    temp = list(tups_corrected[i])
    temp[0] = corrects[i]
    tups_corrected[i] = tuple(temp)

In [19]:
for i in range(len(socio)):
    for j in range(len(tups)):
        if socio.loc[i,'district'] == tups[j][1] and socio.loc[i,'nbhd'] == tups[j][0]:
            socio.at[i,'nbhd'] = tups_corrected[j][0]

In [28]:
socio['district'] = socio['district'].str.replace('EYUP','EYUPSULTAN')

In [49]:
# Joins nbhd id.s to socioeconomic df (socio)
socio = socio.join(nbhds.set_index(['district','neighborhood'])['nbhd_id'], on = ['district', 'nbhd'])

In [52]:
# We already have the needed election data, so the ones in socio df are unnecessary; they are the last columns in dataset
socio.columns[150:]

Index(['Universite Yerleske Sayisi 2017', 'Banka Sube Sayisi 2017',
       'Banka Cesitliligi 2017', 'Banka Sube Basina Dusen Nufus 2017',
       'Satilik Konut Bedeli (m2 TL) 2017',
       'Kiralik Konut Bedeli (m2 TL) 2017',
       'Satilik Konut Bedeli Degisimi (3 yil)',
       'Kiralik Konut Bedeli Degisimi (3 yil)', '2011 Genel Secim 1. Parti',
       '2011 Genel Secimlere Katilma Orani',
       '2011 Genel Secim Gecerli Oy Orani', '2014 Yerel Secim 1. Parti',
       '2014 Yerel Secimlere Katilma Orani %',
       '2014 Yerel Secim Gecerli Oy Orani %',
       '2015 Haziran Genel Secim 1. Parti',
       '2015 Haziran Genel Secimlere Katilma Orani',
       '2015 Haziran Genel Secim Gecerli Oy Orani',
       '2015 Kasim Genel Secim 1. Parti',
       '2015 Kasim Genel Secimlere Katilma Orani',
       '2015 Kasim Genel Secim Gecerli Oy Orani',
       '2015 Kasim Siyasi Homojenlik Endeksi', '2017 Referandum Tercihi',
       'SEÇMENSAYISI_2018_meclis', 'OYKULLANANSEÇMENSAYISI_2018_mecl',


In [57]:
# Drops unncessary columns
socio = socio.drop(socio.loc[:,'2011 Genel Secim 1. Parti':'akp_margin_2014_metropol'], axis=1)

In [109]:
# There is a missing value for one neighborhood population; filled in by male population + female population
socio.at[652,'2016 Toplam Nufus'] = socio.loc[652,'2016 Toplam Nufus (Erkek)'] + socio.loc[652,'2016 Toplam Nufus (Kadin)']

In [61]:
# Creates a sub-list of columns for filtering df with only necessary columns
sub_list = list()

In [70]:
# Explores the first 50 columns' names
socio.columns[:50]

Index(['id', '2016 Toplam Nufus', '2016 Toplam Nufus (Erkek)',
       '2016 Toplam Nufus (Kadin)', '2015 Toplam Nufus',
       '2015 Toplam Nufus (Erkek)', '2015 Toplam Nufus (Kadin)',
       '2014 Toplam Nufus', '2014 Toplam Nufus (Erkek)',
       '2014 Toplam Nufus (Kadin)', 'Nufus Artis Yonu (2014-2016)',
       'Istanbul Nufusu Icerisindeki Pay Degisim Yonu (2014-2016)',
       'Z Kusagi (0-19 yas) 2016 ', 'Y Kusagi (20-39 yas) 2016',
       'X Kusagi (40-54 yas) 2016',
       'Baby Boomer/Geleneksel Kusak (+55 yas) 2016',
       'Z Kusagi (0-19 yas) 2015 ', 'Y Kusagi (20-39 yas) 2015',
       'X Kusagi (40-54 yas) 2015',
       'Baby Boomer/Geleneksel Kusak (+55 yas) 2015',
       'Z Kusagi (0-19 yas) 2014 ', 'Y Kusagi (20-39 yas) 2014',
       'X Kusagi (40-54 yas) 2014',
       'Baby Boomer/Geleneksel Kusak (+55 yas) 2014', 'Ortalama Yas 2016',
       'Ortalama Yas 2015', 'Ortalama Yas 2014', 'Yas Bagimlilik Orani 2016',
       'Yas Bagimlilik Orani 2015', 'Yas Bagimlilik Orani 

In [66]:
# Picks those that will be used for analysis/feature engineering
sub_list.extend(['2016 Toplam Nufus', '2016 Toplam Nufus (Erkek)', '2016 Toplam Nufus (Kadin)', '2015 Toplam Nufus', 'Z Kusagi (0-19 yas) 2016 ', 'Y Kusagi (20-39 yas) 2016', 'X Kusagi (40-54 yas) 2016', 'Ortalama Yas 2016', 'Ortalama Yas 2015', 'Ortalama Yas 2014', 'Hic Evlenmeyenler 2016', 'Evli Olanlar 2016', 'Bosananlar 2016', 'Yuzolcumu (km2)', 'Hane Halki Ortalama Buyuklugu 2016'])

In [71]:
# Explores columns
socio.columns[50:100]

Index(['Toplam Konut Sayisi 2016', 'Yazlik Sayisi 2016',
       'Ozel Isyeri Sayisi 2016', 'Toplam Konut Sayisi 2015',
       'Yazlik Sayisi 2015', 'Ozel Isyeri Sayisi 2015',
       'Toplam Konut Sayisi 2014', 'Yazlik Sayisi 2014',
       'Ozel Isyeri Sayisi 2014', 'Bilinmeyen 2016', 'Okur Yazar Degil 2016',
       'Okur Yazar Fakat Bir Okul Bitirmeyen 2016', 'Ilkokul Mezunu 2016',
       'Ilkogretim Mezunu 2016', 'Ortaokul Mezunu 2016', 'Lise Mezunu 2016',
       'Universite Mezunu 2016', 'Yuksek Lisans Mezunu 2016',
       'Doktora Mezunu 2016', 'Bilinmeyen 2015', 'Okur Yazar Degil 2015',
       'Okur Yazar Fakat Bir Okul Bitirmeyen 2015', 'Ilkokul Mezunu 2015',
       'Ilkogretim Mezunu 2015', 'Ortaokul Mezunu 2015', 'Lise Mezunu 2015',
       'Universite Mezunu 2015', 'Yuksek Lisans Mezunu 2015',
       'Doktora Mezunu 2015', 'Bilinmeyen 2014', 'Okur Yazar Degil 2014',
       'Okur Yazar Fakat Bir Okul Bitirmeyen 2014', 'Ilkokul Mezunu 2014',
       'Ilkogretim Mezunu 2014', 'Ortao

In [72]:
# Picks useful ones
sub_list.extend(['Toplam Konut Sayisi 2016', 'Ozel Isyeri Sayisi 2016', 'Ilkokul Mezunu 2016', 'Universite Mezunu 2016', 'Okuryazar Orani - Kadin 2016', 'Ortalama Egitim Suresi 2016'])

In [77]:
# Explores columns
socio.columns[100:150]

Index(['Ortalama Egitim Suresi - Erkek 2016',
       'Ortalama Egitim Suresi - Kadin 2016', 'Ortalama Egitim Suresi 2015',
       'Ortalama Egitim Suresi 2014', 'Ilkokul - Toplam Ogrenci Sayisi',
       'Ilkokul - Toplam Sube Sayisi', 'Ilkokul - Toplam Ogretmen Sayisi',
       'Ilkokul - Ogretmen Basina Dusen Ogrenci Sayisi',
       'Ilkokul - Sube Basina Dusen Ogrenci Sayisi',
       'Ortaokul - Toplam Ogrenci Sayisi', 'Ortaokul - Toplam Sube Sayisi',
       'Ortaokul - Toplam Ogretmen Sayisi',
       'Ortaokul - Ogretmen Basina Dusen Ogrenci Sayisi',
       'Ortaokul - Sube Basina Dusen Ogrenci Sayisi',
       'Lise - Toplam Ogrenci Sayisi', 'Lise - Toplam Sube Sayisi',
       'Lise - Toplam Ogretmen Sayisi',
       'Lise - Ogretmen Basina Dusen Ogrenci Sayisi',
       'Lise - Sube Basina Dusen Ogrenci Sayisi',
       'Toplam Ogrenci Sayisi (Ilk-Orta-Lise)', 'ASM Varligi',
       'ASM Doktor Sayisi', 'ASM Saglik Personeli Sayisi',
       'ASM Doktor Basina Ortalama Muayene Sayisi', '

In [78]:
# Picks useful ones
sub_list.extend(['ASM Varligi', 'Ozel Hastane Varligi', 'Kultur Sanat Merkezi Varligi', 'Sinema Varligi', 'Tiyatro Varligi', 'Dogalgaz Kullanici Sayisi/Nufus 2017', 'Su Ariza Suresi (Saat) 2016', 'Metro Durak Sayisi', 'Market Yogunluk Endeksi 2017', 'AVM Sayisi 2017', 'Pazar Yeri Sayisi 2017'])

In [80]:
# Explores columns
socio.columns[150:]

Index(['Universite Yerleske Sayisi 2017', 'Banka Sube Sayisi 2017',
       'Banka Cesitliligi 2017', 'Banka Sube Basina Dusen Nufus 2017',
       'Satilik Konut Bedeli (m2 TL) 2017',
       'Kiralik Konut Bedeli (m2 TL) 2017',
       'Satilik Konut Bedeli Degisimi (3 yil)',
       'Kiralik Konut Bedeli Degisimi (3 yil)', 'district', 'nbhd', 'nbhd_id'],
      dtype='object')

In [81]:
# Picks useful ones
sub_list.extend(['Kiralik Konut Bedeli (m2 TL) 2017','Satilik Konut Bedeli (m2 TL) 2017'])

In [84]:
# Adds the identification column names to the list-to-filter
sub_list = ['nbhd_id','district','nbhd'] + sub_list

In [113]:
# Filters df to keep only the necessary columns
socio_filtered = socio[sub_list]

In [143]:
# Initiates list for filtering rows
rows_to_ignore = list()

In [144]:
# Sile, Catalca and Silivri are the three rural districts of Istanbul
# Most places that are officially listed as neighborhoods in these districts are in fact villages
# These 3 lists are those neighborhoods that comprise the city center of the corresponding districts
sile_merkez = ['BALIBEY', 'CAVUS', 'CAYIRBASI', 'HACI KASIM', 'KUMBABA']
catalca_merkez = ['FERHATPASA', 'KALEICI']
silivri_merkez = ['SELIMPASA', 'KAVAKLI', 'HURRIYET', 'YENI', 'CUMHURIYET', 'ALIBEY', 'FATIH', 'PIRI MEHMET PASA', 'MIMAR SINAN', 'SEMIZKUMLAR']

In [146]:
# Adds the non-central neighborhoods' (villages') nbhd_id's to the rows_to_ignore list
for row in nbhds[(nbhds['district'] == 'SILE')].itertuples():
    if row[2] not in sile_merkez: 
        rows_to_ignore.append(row[4])
        
for row in nbhds[(nbhds['district'] == 'CATALCA')].itertuples():
    if row[2] not in catalca_merkez: 
        rows_to_ignore.append(row[4])
        
for row in nbhds[(nbhds['district'] == 'SILIVRI')].itertuples():
    if row[2] not in silivri_merkez: 
        rows_to_ignore.append(row[4])

In [151]:
# Retains only the non-village neighborhoods in the filtered df socio_filtered
socio_filtered = socio_filtered[~socio_filtered['nbhd_id'].isin(rows_to_ignore)]

In [340]:
# Filters neighborhoods with population above 500, for consistent and stable statistical analyses
socio_filtered = socio_filtered[socio_filtered['2019 Toplam Nufus'] > 500]

In [349]:
# We see that only a few null entries exist, so we can safely drop them and work with the remaining ones
socio_filtered = socio_filtered.reset_index()
socio_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 0 to 958
Data columns (total 39 columns):
nbhd_id                                 804 non-null int64
district                                804 non-null object
nbhd                                    804 non-null object
2016 Toplam Nufus                       804 non-null object
2016 Toplam Nufus (Erkek)               804 non-null object
2016 Toplam Nufus (Kadin)               804 non-null object
2015 Toplam Nufus                       804 non-null float64
Z Kusagi (0-19 yas) 2016                804 non-null object
Y Kusagi (20-39 yas) 2016               804 non-null object
X Kusagi (40-54 yas) 2016               804 non-null object
Ortalama Yas 2016                       804 non-null object
Ortalama Yas 2015                       804 non-null object
Ortalama Yas 2014                       804 non-null object
Hic Evlenmeyenler 2016                  804 non-null object
Evli Olanlar 2016                       804 non-null ob

In [350]:
socio_filtered = socio_filtered.dropna()

In [371]:
# Function that calculates rate of change from previous year to next
def rate_of_change(prev_yr, next_yr):
    return (next_yr - prev_yr) / prev_yr

In [355]:
# Creates population change features for the years that exist in our dataset
socio_filtered['pop change 15-16'] = rate_of_change(socio_filtered['2015 Toplam Nufus'],socio_filtered['2016 Toplam Nufus'])
socio_filtered['pop change 16-17'] = rate_of_change(socio_filtered['2016 Toplam Nufus'],socio_filtered['2017 Toplam Nufus'])
socio_filtered['pop change 17-18'] = rate_of_change(socio_filtered['2017 Toplam Nufus'],socio_filtered['2018 Toplam Nufus'])
socio_filtered['pop change 18-19'] = rate_of_change(socio_filtered['2018 Toplam Nufus'],socio_filtered['2019 Toplam Nufus'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [366]:
# Drops total population features except the most recent one, since keeping them would cause collinearity issues
socio_filtered = socio_filtered.drop(columns=['2015 Toplam Nufus','2016 Toplam Nufus','2017 Toplam Nufus','2018 Toplam Nufus'])

In [368]:
# Creates a feature for female population per male population
socio_filtered['women:men'] = socio_filtered['2016 Toplam Nufus (Kadin)'] / socio_filtered['2016 Toplam Nufus (Erkek)']

In [369]:
# Drops gender population features
socio_filtered = socio_filtered.drop(columns=['2016 Toplam Nufus (Kadin)', '2016 Toplam Nufus (Erkek)'])

In [383]:
# Creates rate of change feature for average age
socio_filtered['avg age change 15-16'] = rate_of_change(socio_filtered['Ortalama Yas 2015'],socio_filtered['Ortalama Yas 2016'])

In [384]:
# Drops the older average age feature
socio_filtered = socio_filtered.drop(columns=['Ortalama Yas 2015'])

In [391]:
# Population density is calculated
socio_filtered['population density per km2'] = socio_filtered['2019 Toplam Nufus'] / socio_filtered['Yuzolcumu (km2)']

In [408]:
# As number of features feature and total population are highly correlated, a new feature that reduces multicollinearity is engineered
socio_filtered['Residences per Population'] = socio_filtered['Toplam Konut Sayisi 2016'] / socio_filtered['2019 Toplam Nufus']

In [410]:
socio_filtered = socio_filtered.drop(columns=['Toplam Konut Sayisi 2016'])

In [564]:
socio_filtered.columns

Index(['index', 'nbhd_id', 'district', 'nbhd', 'Z Kusagi (0-19 yas) 2016 ',
       'Y Kusagi (20-39 yas) 2016', 'X Kusagi (40-54 yas) 2016',
       'Ortalama Yas 2016', 'Hic Evlenmeyenler 2016', 'Evli Olanlar 2016',
       'Bosananlar 2016', 'Yuzolcumu (km2)',
       'Hane Halki Ortalama Buyuklugu 2016', 'Ozel Isyeri Sayisi 2016',
       'Ilkokul Mezunu 2016', 'Universite Mezunu 2016',
       'Okuryazar Orani - Kadin 2016', 'Ortalama Egitim Suresi 2016',
       'ASM Varligi', 'Ozel Hastane Varligi',
       'Dogalgaz Kullanici Sayisi/Nufus 2017', 'Su Ariza Suresi (Saat) 2016',
       'Metro Durak Sayisi', 'Market Yogunluk Endeksi 2017', 'AVM Sayisi 2017',
       'Pazar Yeri Sayi 2017', '2019 Toplam Nufus', 'pop change 16-17',
       'pop change 17-18', 'pop change 18-19', 'pop change 15-16', 'women:men',
       'avg age change 15-16', 'population density per km2',
       'Residences per Population', 'cultural high', 'cultural med',
       'real estate', 'real estate low', 'real estate h

In [447]:
socio_filtered['cultural'] = socio_filtered['Kultur Sanat Merkezi Varligi'] + socio_filtered['Sinema Varligi'] + socio_filtered['Tiyatro Varligi']

In [455]:
socio_filtered['cultural'] = socio_filtered['cultural'].replace(3,2)

In [463]:
socio_filtered['cultural high'] = 0
socio_filtered['cultural med'] = 0

for i in range(len(socio_filtered)):
    if socio_filtered.loc[i,'cultural'] == 2:
        socio_filtered.at[i,'cultural high'] = 1
    elif socio_filtered.loc[i,'cultural'] == 1:
        socio_filtered.at[i,'cultural med'] = 1

In [467]:
socio_filtered = socio_filtered.drop(columns=['cultural', 'Kultur Sanat Merkezi Varligi', 'Sinema Varligi', 'Tiyatro Varligi'])

In [477]:
socio_filtered['Metro Durak Sayisi'].unique()

array([0., 1., 2., 3., 5., 4.])

In [478]:
socio_filtered['Metro Durak Sayisi'] = socio_filtered['Metro Durak Sayisi'].replace({5:1, 4:1, 3:1, 2:1})

In [498]:
socio_filtered['AVM Sayisi 2017'] = socio_filtered['AVM Sayisi 2017'].replace({4:1, 3:1, 2:1})

In [528]:
not_zero = [int(i) for i in socio_filtered['Pazar Yeri Sayi 2017'].unique() if i != 0 and i != 1]

replace_dict = dict()
for k in not_zero:
    replace_dict[k] = 1

socio_filtered['Pazar Yeri Sayi 2017'] = socio_filtered['Pazar Yeri Sayi 2017'].replace(replace_dict)

In [553]:
socio_filtered = socio_filtered.join(socio.set_index('nbhd_id')['Satilik Konut Bedeli (m2 TL) 2017'], on='nbhd_id')

In [554]:
socio_filtered['real estate'] = (socio_filtered['Satilik Konut Bedeli (m2 TL) 2017'] + socio_filtered['Kiralik Konut Bedeli (m2 TL) 2017']) / 2

In [557]:
socio_filtered['real estate low'] = 0
socio_filtered['real estate high'] = 0

for i in range(len(socio_filtered)):
    if socio_filtered.loc[i,'real estate'] >= 6.5:
        socio_filtered.at[i,'real estate high'] = 1
    elif 4 > socio_filtered.loc[i,'real estate'] > 0:
        socio_filtered.at[i,'real estate low'] = 1

In [562]:
socio_filtered = socio_filtered.drop(columns=['real estate','Kiralik Konut Bedeli (m2 TL) 2017','Satilik Konut Bedeli (m2 TL) 2017'])

In [566]:
socio_filtered

,index,nbhd_id,district,nbhd,Z Kusagi (0-19 yas) 2016,Y Kusagi (20-39 yas) 2016,X Kusagi (40-54 yas) 2016,Ortalama Yas 2016,Hic Evlenmeyenler 2016,Evli Olanlar 2016,...,pop change 18-19,pop change 15-16,women:men,avg age change 15-16,population density per km2,Residences per Population,cultural high,cultural med,real estate low,real estate high
0,0,101,ADALAR,BURGAZADA,22.5,23.98,11.95,46.56,25.65,54.71,...,-0.015183,-0.0097493,1.0431,0.0128345,880.864198,1.700771,0,1,0,0
1,1,102,ADALAR,HEYBELIADA,21.19,30.45,15.28,42.25,32.54,50.34,...,-0.059903,-0.214838,0.922723,0.158169,1629.501916,0.903833,0,0,0,1
2,2,103,ADALAR,KINALIADA,21.26,22.99,10.6,48.04,24.73,62.13,...,-0.009577,-0.0693698,0.914573,0.00523122,1292.647059,2.192264,0,0,1,0
3,3,104,ADALAR,MADEN,22.72,28.54,19.4,40.47,28.2,55.46,...,-0.057671,-0.00482094,0.948315,0.00696691,1732.000000,0.949423,0,0,0,1
4,4,105,ADALAR,NIZAM,23.64,26.28,19.32,41.36,27.55,56.19,...,-0.081038,0.0200067,0.973548,0.00681597,1074.303406,1.190202,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,954,3909,ZEYTINBURNU,SUMER,21.27,34.19,30.31,32.54,29.95,59.6,...,0.029979,-0.0197768,0.999092,0.00930521,56896.969697,0.340301,0,0,0,0
794,955,3910,ZEYTINBURNU,TELSIZ,20.07,35.4,30.89,31.94,31.61,59.88,...,0.023408,-0.0107323,0.983924,0.00852542,62448.333333,0.315861,0,0,0,0
795,956,3911,ZEYTINBURNU,VELIEFENDI,19.08,36.15,30.86,31.69,33.18,58.3,...,0.016965,0,0.970516,0.00763116,68187.500000,0.299762,0,0,0,0
796,957,3912,ZEYTINBURNU,YENIDOGAN,20.35,34.89,29.5,32.7,33.01,56.24,...,0.053921,0.0220547,0.928508,-0.00426309,70731.250000,0.296103,0,0,0,0


## References

[1] https://www.tbmm.gov.tr/kanunlar/k6360.html